In [ ]:
!pip install -U tensorflow-addons

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from keras import layers

In [ ]:
num_classes = 100
input_shape = (32, 32, 3)

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i])
plt.show()

In [ ]:
patch_size = (2, 2)  # 2-by-2 sized patches
dropout_rate = 0.03  # Dropout rate
num_heads = 8  # Attention heads
embed_dim = 64  # Embedding dimension
num_mlp = 256  # MLP layer size
qkv_bias = True  # Convert embedded patches to query, key, and values with a learnable additive value
window_size = 2  # Size of attention window
shift_size = 1  # Size of shifting window
image_dimension = 32  # Initial image size

num_patch_x = input_shape[0] // patch_size[0]
num_patch_y = input_shape[1] // patch_size[1]

learning_rate = 1e-3
batch_size = 128
num_epochs = 40
validation_split = 0.1
weight_decay = 0.0001
label_smoothing = 0.1

In [ ]:
# Xây dựng phương thức tạo vách ngăn cửa số cho hình ảnh 
def window_partition (x , window_size):
    # lấy ra các kích thước của 1 tensor đầu vào x 
    # gồm batch_size , h , w , c 
    _ , height , width , channels = x.shape 
    # lấy ra số lượng bản vá theo chiều x , y bằng cách chia 
    # 2 chiều cho kích thước cửa sổ 
    patch_num_x = width // window_size 
    patch_num_y = height // window_size 
    # Định hình lại x thành hình dạng  (x là 1 tensor)
    # -1 số lượng hình ảnh , số lượng patch theo chiều y , chiều cao của patch
    #  số lượng patch_theo x , chiều ngang của patch , số kênh màu 
    x = tf.reshape(
        x , shape=(-1 , patch_num_y , window_size , patch_num_x , window_size , channels)
    )
    # sau đó chuyển vị lại hình ảnh 
    # với shape = [-1 , patch_num_y , patch_num_x , window_size , window_size , channels]
    x = tf.transpose(x , (0 , 1 , 3 , 2, 4 ,5 ))
    # định dạng lại kích thước của các cứa sổ và trả về nó 
    # shape = [số lượng cửa sổ , x , y , c]
    windows = tf.reshape(x , shape=(-1 , window_size , window_size , channels))
    return windows 

# Xây dựng hàm cứa sổ đảo ngược dùng để ghép các bản vá thành bản vá lớn hơn 
# hàm này có chức năng ngược lại với hàm tạo vách ngăn cửa sổ 
def window_reverse(windows, window_size, height, width, channels):
    # Tính toán số lượng bản vá theo trục x và y 
    patch_num_y = height // window_size
    patch_num_x = width // window_size 
    # định hình lại x thành dạng (x là 1 tensor)
    # (số lượng patch_theo x, số lượng patch theo x, theo y , kích thước 2  chiều của mỗi patch , kênh màu)
    x = tf.reshape(
        windows , 
        shape =(-1  , patch_num_y , patch_num_y , window_size , window_size , channels)
    )
    # sau đó chuyển vị tensor này về dạng 
    # sahpe = (số lượng , patch_num_y , chiều cao patch (window_size) , patch_num_x (chiều nagng)
    # só kênh màu (channels))
    x = tf.transpose(x, perm=(0 ,1 ,3 ,2 ,4 ,5))
    # định hình lại kích thước cho x với -  1 là tham số tự tính cho phù hợp 
    # với bước tính toán để có được số lượng cửa sổ phù hợp 
    x = tf.reshape(x , shape=(-1 , height , width , channels))
    return x 

# Xây dựng hàm Dropath để loại bỏ ngẫu nhiên cho một tensor đầu vào 
layers.Dropout(0.1) 
class DropPath(layers.Layer):
    def __init__(self, drop_prob=None , **kwargs):
        super().__init__(**kwargs)
        self.drop_prob = drop_prob 

    def call(self, x):
        # lấy ra kích thước của tensor đầu vào x 
        input_shape = tf.shape(x)
        # lấy ra kích thước lô 
        batch_size = input_shape[0]
        # lấy ra số chiều của x bằng hàm rank 
        rank = x.shape.rank #  = 4 shape x [batch_size , window_size , widow_size , channels]
        # tạo 1 biến shape có shape = batch_size , 1 , 1 ,1 
        # đầu tiên ta tạo ma biến typle với batch_size phàn tử là chiều đầu tiên của shape 
        # sau đó ta tính toán số chiều còn lại  = 1 *(rank-1) tức là 3 chiều 
        # với shape 3 chiều  = 1 
        shape = ( batch_size,) + (1,) * (rank-1) # shape = [batch_size , 1 , 1 ,1]
        # sau đó tạo 1 tensor ngẫu nhiên = xác xuất 1 - drop_prob  + shape 
        # mục đích tạo ra 1 tensor với các phần tử đc lấy ngẫu nhiên [0 -> 1]
        random_tensor = (1 - self.drop_prob) + tf.random.uniform(shape , dtype=x.dtype)
        # Xây dựng một ma trận Path_mask bằng cách làm tròn xuống các tỷ lệ của tensor 
        path_mask = tf.floor(random_tensor)
        # sau đó tính đầu ra bằng thực hiện chia cho tỷ lệ 1 - drop_prob rồi nhân với ma trận 
        # tỷ lệ path_mask 
        output = tf.math.divide(x , 1 - self.drop_prob) * path_mask
        return output 
    



In [ ]:
# Xây dựng Của sổ chú ý cho khối swin transformer Block 
# với các tham số , dim (số chiều không gian vector) , window_size , num_heads ,
# dropout_rate 
class WindowAttention(layers.Layer):
    def __init__(
            self, dim , window_size , num_heads , qkv_bias=True , dropout_rate=0.0, **kwargs
    ):
        super().__init__(**kwargs)
        self.dim = dim 
        self.window_size = window_size
        self.num_heads = num_heads
        # tạo một hệ số tỷ lệ  = _/ dim // num_heads
        # sử dụng cho phép tính toán self-attention 
        self.scale = (dim // self.num_heads) ** -0.5
        self.qkv = layers.Dense(dim*3 , use_bias=qkv_bias)
        self.dropout = layers.Dropout(dropout_rate)
        # thêm một lớp nhúng tuyến tính 
        self.proj = layers.Dense(dim)

    # Xây dựng phương thức sử dụng để tính toán các chỉ số vị trị tương đối 
    # cho mỗi cặp phần tử trong cửa sổ 
    def build (self, input_shape):
        # Khởi tạo ma trận chứa số lượng phần tử trong cửa sổ 
        num_window_elements = (2 * self.window_size[0]- 1) * (
            2 * self.window_size [1] - 1
        )
        # Xây dựng bảng chứa các giá trị bias vị trí tương đối 
        # cho mỗi cặp các phần tử trong các cứa sổ 
        # shape = [num_elemnet , num _heads]
        self.relative_position_bias_table = self.add_weight(
            shape =(num_window_elements , self.num_heads) , 
            # đặt tất acr các tham số  trong ma bảng = 0 và 
            # cho phép cập nhật nó trong quá trình huấn luyện
            initializer= tf.initializers.Zeros(),
            trainable = True,
        )
        # khởi tạo 2 mảng chứa các chỉ số hàng và cột của các phần tử trong cửa sổ .
        coords_h =  np.arange(self.window_size[0])
        coords_w =  np.arange(self.window_size[1])
        # Xây dựng ma trận coords từ ma trận hàng và cột ở trênn
        # đặt indexing = ij để mỗi phần tủ trong ma trận là duy nhất
        coords_matrix = np.meshgrid(coords_h , coords_w , indexing='ij')
        # xây dựng  tensor coords bằng cách xếp trồng ma trận coords 
        # shap = [2 , window_size[0] , window_size[1]]  chiều đầu tiên là chiều của các chỉ số 
        # hàng và cột
        coords = np.stack(coords_matrix)
        # sau flatten lại tensor này thành 2 chiều 
        # shape = [2 , -1]  với -1 là chỉ số tự tính  = num_window_element 
        coords_flatten = coords.reshape(2, -1)
        # Xây dựng ma trận relative_coords  bằng cách thêm chiều cho ma trận flattent
        # sau đó thực hiện phép trừ để có được ma trận relative_coords 
        # shape = [2, num_window_elements, num_window_elements] 
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :,]
        # Sau đó thực hiện chuyển vị ma trận shape = [num_window_elements, num_window_elements, 2]
        # Chiều cuối cùng cho biết khoảng cách vị trí hàng và cột 
        relative_coords = relative_coords.transpose([1 , 2 , 0])
        # Cộng các chỉ số khoảng cách theo hàng và cột của ma trận với một hằng số duy nhất 
        relative_coords[:, :, 0] += self.window_size[0] -1 
        relative_coords[:, :, 1] += self.window_size[1] - 1
        # Nhân các chỉ só vị trí tương đối của ma trận với hằng số để có được giá trị duy nhất 
        # cho mỗi hàng 
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        # Cộng hai chỉ số hàng và cột cửa ma trận relative_coords theo chiều cuối để thu được 
        # một ma trận chứa các cặp chỉ số tương đối cho mỗi cặp phần tử trong cửa sổ 
        # shape = [num_elements, num_elements]
        relative_position_index = relative_coords.sum(-1)

        # Xây dựng ma trận chỉ số vị trí tương đối cho mỗi cặp phần tử trong cửa sổ
        # và accs chỉ số này không được cập nhật
        self.relative_position_index = tf.Variable(
            initial_value=tf.convert_to_tensor(relative_position_index), 
            trainable=False,
        )

    # Xây dựng phương thức tính toán Attention , ma trận position_bias , và masked attention
    def call(self, x,  mask = None):
        # lấy ra kích thước của tensor x
        _ , size , channels  = x.shape 
        head_dim = channels // self.num_heads
        # Tính toán qkv bằng cách ánh xạ x qua lớp mạng dày đặc 
        x_qkv = self.qkv(x)
        # thay đổi hjinhf dạng của tensor x  = [ batch_size , size , 3, heads , head_dim]
        x_qkv = tf.reshape(x_qkv, shape=(-1, size, 3, self.num_heads, head_dim))
        # Chuyển vị x_qkv = [3 , batch_size , heads , size , head_dim] để phân biệt được 
        # 3 vector q , k , v
        x_qkv = x_qkv = tf.transpose(x_qkv, perm=(2, 0, 3, 1, 4))
        # lấy ra q , k , v lần lượt 
        q, k, v = x_qkv[0], x_qkv[1], x_qkv[2]
        # sau đó nhân q với hệ só scale để tăng tính đa dạng cho q
        q = q *self.scale 
        # Thực hiện chuyển vị k để có thể nhân k với q 
        # shape = [batch_size , num_heads , head_dim , size]
        k = tf.transpose(k , perm=(0 ,1 ,3 ,2))
        # Nhân ma trận q vs  k để thu được tensor 
        # shape = [batch_size,  num_heads , size ,size]
        attn = q @ k 


        # Tính toán só lượng phần tử trong một cửa sổ 
        num_window_elements = self.window_size[0] * self.window_size[1]
        # Thay đổi hình dạng của tensor relative_position_idex 
        # thành ma trận 1 chiều duy nhất chứa các chỉ số idx của các cặp phần tử
        relative_position_index_flat = tf.reshape(
            self.relative_position_index, shape=(-1,)
        )
        # lấy các giá trị bias cho các vị trí tương đối từ bảng relative_position_bias
        #  theo chỉ số relative_position_index_flat
        relative_position_bias = tf.gather(
            self.relative_position_bias_table, relative_position_index_flat
        )
        # Định hình lại hình dạng tensor thanh [num_elements , num_elements , num heads]
        relative_position_bias = tf.reshape(
            relative_position_bias, shape=(num_window_elements, num_window_elements, -1)
        )
        # chuyển vị ma trận relative_position_bias shape []
        relative_position_bias = tf.transpose(relative_position_bias, perm=(2, 0, 1))
        # sau đó công ma attn vs relative_position_bias 
        # để thêm bias cho các vị trí tương đối trong cửa sổ # chú ý 2 ma trận này có cùng kích thước như nhau khác chỉ số batch_size 
        #  S sẽ có dạng (batch_size, num_heads, size, size)
        attn = attn + tf.expand_dims(relative_position_bias, axis=0)

        # Xây dựng mặt nạ cho window attention 
        # Kiểm tra xem Mask có tồn tại không là 1 tensor nhị phân
        if mask is not None :
            # lấy ra kích thước thứ nhất của mask là số lượng cửa sổ  trong x
            num_Window = mask.get_shape()[0]
            # chuyển mask thành kiểu số thực và thêm 2 chiều rỗng 
            # shap = (1, 1, num_windows, window_size * window_size, window_size * window_size)
            mask_float = tf.cast(
                tf.expand_dims(tf.expand_dims(mask, axis=1), axis=0), tf.float32
            )
            # thay đổi hình dnagj của attention , thêm số lượng cửa sổ vào attent 
            # sau đó cộng mask choa atention để che đi các chỉ số 
            # shape attn = -1 , num_Window , self.num_heads , size , size
            attn = (
                # [batch_size , num_window ,num head, size , size]
                tf.reshape(attn, shape=(-1 , num_Window , self.num_heads , size , size))
                # + (1, 1, num_windows, window_size * window_size, window_size * window_size)
                + mask_float
            )
            # Sau khi thêm mặt nạ attention ta trả attention về kích thước ban đầu 
            attn = tf.reshape(attn, shape=(-1 , self.num_heads , size, size))
            # áp dụng tính toán softmax cho attention theo chiều cuối tức size 
            # là kích thước theo chiều của ma trận 
            attn = keras.activations.softmax(attn, axis=-1)
        else:
            # áp dụng tính toán softmax cho attention theo chiều cuối tức size ngay lập tức
            attn = keras.activations.softmax(attn, axis=-1)
        # loại bỏ bớt các tham số có tỷ lệ kém 
        attn = self.dropout(attn)

        # Tính điểm socre cho attention shap = [ batch_size , num_heads , size , head_dim]
        x_qkv = attn @ v
        # Hoán vị các chiều của x_qkv để có dạng (batch_size, size, num_heads, head_dim)
        x_qkv = tf.transpose(x_qkv, perm=(0, 2, 1, 3))
        # Thay đổi hình dạng của x_qkv thành (batch_size, size, channels), 
        # để nối các đầu chú ý lại với nhau theo chiều thứ 3 
        x_qkv = tf.reshape(x_qkv, shape=(-1, size, channels))
        # áp dụng lớp dense để biến đổi về 1 tensor có kích thước ban đầu và 1 lớp bỏ học 
        x_qkv = self.proj(x_qkv)
        x_qkv = self.dropout(x_qkv)
        return x_qkv


In [ ]:
# Xây dựng khối swin transformer 
class SwinTransformer(layers.Layer):
    def __init__(
        self,
        dim,
        num_patch,
        num_heads,
        window_size=7,
        shift_size=0,
        num_mlp=1024,
        qkv_bias=True,
        dropout_rate=0.0,
        **kwargs,
    ):
        super().__init__(**kwargs)

        self.dim = dim  # number of input dimensions
        self.num_patch = num_patch  # number of embedded patches
        self.num_heads = num_heads  # number of attention heads
        self.window_size = window_size  # size of window
        self.shift_size = shift_size  # size of window shift
        self.num_mlp = num_mlp  # number of MLP nodes

        # Xây dựng lớp layerNormalization 
        self.norm1 = layers.LayerNormalization(epsilon=1e-5)
        # Lớp attention 
        self.attn = WindowAttention(
            dim , 
            window_size=(self.window_size , window_size),
            qkv_bias=qkv_bias,
            num_heads=num_heads,
            dropout_rate=dropout_rate
        )
        self.drop_path = DropPath(dropout_rate)
        self.norm2 = layers.LayerNormalization(epsilon=1e-5)

        self.mlp = keras.Sequential(
            [
                layers.Dense(num_mlp),
                layers.Activation(keras.activations.gelu),
                layers.Dropout(dropout_rate),
                layers.Dense(dim),
                layers.Dropout(dropout_rate),
            ]
        )
        if min(self.num_patch) < self.window_size:
            self.shift_size = 0
            self.window_size = min(self.num_patch)
        
    #  Xây dựng phương thức thực hiện Shifted Window Attention 
    def build(self, input_shape):
        # Kiểm tra xem shift size có bằng  
        if self.shift_size == 0 :
            # nếu = 0 không cần xây dựng mặt nạ 
            self.attn_mask = None 
        # xét trường hợp còn lại
        else:
            # lấy ra chiều h và w từ num_patch
            # num_patch là một ma trận chứa số lượng bản vá h , w là số lượng bản vá theo 
            # chiều tương ứng 
            height , width = self.num_patch
            # xây dựng lát cắt theo chiều cao 
            # mỗi lát cắt dài = window_size và được dịch chuyển với 
            # bước nhảy = shift_size 
            h_slices = (
                # mỗi trục gồm 3 lát cắt 
                slice(0 , - self.window_size),
                slice(-self.window_size, -self.shift_size),
                # lát cắt cuối trượt từ - 1 đến hết 
                slice(-self.shift_size, None),
            )
            # xây dựng lát cắt theo chiều cao 
            # mỗi lát cắt dài = window_size và được dịch chuyển với 
            # bước nhảy = shift_size
            w_slices = (
                slice(0, -self.window_size),
                slice(-self.window_size, -self.shift_size),
                slice(-self.shift_size, None),
            )
            # Xây dựng một mặt nạ ma trận = 0 shape = [1 , h , w , 1]
            mask_array = np.ones((1 , height , width , 1))
            # khởi tạo biến count cho biết số lượng cửa sổ đã duyệt qua 
            count = 0 
            # duyệt qua các lát cắt của slice
            for h in h_slices:
                for w in w_slices:
                    # sau đó gán cho mỗi phần tử trong mảng từ 0 -> 8 theo count 
                    mask_array[:, h , w , :] = count 
                    count += 1
            # chuyển đổi mask_array thành 1 tensor 
            mask_array = tf.convert_to_tensor(mask_array)
            # Xây dựng ma trận mặt nạ cho cửa sổ 
            # phân chia mask_array thành các cửa sổ nhỏ hơn có kích thước window_size
            # trả về 1 tensor mỗi phần tử của tensor này  giá trị 0- > nố lượng patch trong 1 ô 
            # tương ứng với vị trị của cửa sổ trong mội khối 
            mask_windows = window_partition(mask_array , self.window_size)
            # Reshape lại tensor với shape = [-1 , window_size * window_size]
            # tức là mỗi hàng là một cứa sổ 
            mask_windows = tf.reshape(
                mask_windows , shape=[-1 , self.window_size * self.window_size]
            )
            # Tính toán Attention_mask bằng cách lấy hiệu giữa các hàng của tensor 
            # shape = [num_window , window_size * window_size ,  window_size * window_size]
            # mỗi phần tử trong tensor là 1 ma trận vuông shape = [window_size * window_size * window_size * window_size]
            # biểu diễn mức độ tương qua giữa các patch trong một cửa sổ với các patch trong cửa sổ khác 
            attn_mask = tf.expand_dims(mask_windows , axis=1) - tf.expand_dims(mask_windows , axis=2)
            # Thay đổi giá trị trong attn_mask theo hiệu của mask_windows 
            # nếu hai hàng có giá trị khác nhau thì không trùng = -100 cần tính attention 
            attn_mask = tf.where(attn_mask != 0, -100.0 ,attn_mask )
            # nếu 2 hàng bằng nhau gán  = 0.0 không cần tính attention 
            attn_mask = tf.where(attn_mask == 0, 0.0, attn_mask)
            # Khởi tạo att_mask là một biến không thay đổi
            # shape = [num_window * window_size * window_size * window_size * window_size]
            self.attn_mask = tf.Variable(initial_value=attn_mask, trainable=False)


    # Xây dựng phương thức thực hiện cơ chế chú ý cửa sổ trên dữ liệu đầu vào 
    def call(self, x): # x là dữ liệu đầu vào shape = batch_size , num_patch , channels 
        # lấy ra chiều cao và chiều rộng từ ma trận num_patch 
        height , width = self.num_patch 
        # lấy ra các kích thước cửa x 
        _ , num_patches_before , channels = x.shape 
        # tạo một biến x_skip để lưu trữ lại x ban đầu , sử dụng cho phép cộng dư thừa 
        # sau này 
        x_skip = x
        # Áp dụng một lớp chuẩn hóa theo tầng cho lớp Mục đích để x có phân bố nhất quán 
        # không bị ảnh hưởng quá nhiều bởi các giá trị ngoại lai. Giúp cho việc xây dựng 
        # cơ chế chú ý cửa sổ cho x sau đó được hiệu quả và ổn định 
        x = self.norm1(x)
        # reshape lại x với kích thước [ batch_size , height , width , channels]
        # để có thể thực hiện cửa sổ trượt cho x 
        x = tf.reshape(x , shape=(-1 , height, width , channels))
        # kiểm tra xem kích thước chuyển đổi shifted_size có > 0 
        # nếu có ta thực hiện cửa sổ dịch chuyển cho ma trận x 
        if self.shift_size > 0:
            # ta thực hiện dịch chuyển cửa sổ theo 2 chiều ngang và dọc
            # tức là ta áp dụng trên chiều 1 và 2 của x 
            shifted_x = tf.roll(
                x , shift=[-self.shift_size , -self.shift_size], axis=[1,2]
            )
        # trường hợp còn lại tức là không tồn tại kích thuớc chuyển đổi 
        else:
            # ta gán shift = x
            shifted_x = x 
        # sau khi thực hiện phương pháp cửa sổ dịch chuyển ta tiến hành tách các 
        # ô cửa sổ trong không gian dịch chuyển thành các vách ngăn cửa sỏ rồi sau đó 
        # thực hiện mặt nạ và tính toán chú ý trên các ô cửa sổ 
        x_windows = window_partition(shifted_x , self.window_size)
        # định hình lại x_windows thành [batch_size , window_size * window_size, channels]
        # để có thể áp dụng lớp chú ý vào 
        x_windows = tf.reshape(x_windows , shape=(-1 , window_size * window_size, channels))
        # Thực hiện tính toán attention cho các ô cửa sổ và áp dụng mặt nạ cho các ô 
        attn_windows = self.attn(x_windows , mask=self.attn_mask)
        # thay đổi hình dạng attn_windows thành [batch_size , window_size, window_size , channels]
        # để có thể chuyển đổi lại ma trận ban đầu 
        attn_windows = tf.reshape(
            attn_windows, shape=(-1, self.window_size, self.window_size, channels)
        )
        # sau đó chuyển đổi lại thành ma trận ban đầu bằn phương thức window_reverse
        # và chuyền vào các tham số tương ứng mục tiếu => shape [batch_size , height , weight , channels]
        shifted_x = window_reverse(
            attn_windows , self.window_size , height , width , channels 
        )
        # nếu shifted_size > 0 thực hiện phép chuyển dịch ngược lại thành hình ảnh ban đầu 
        # theo chiều ngang và dọc 
        if self.shift_size > 0 : 
            x = tf.roll(
                shifted_x , shift=[self.shift_size , self.shift_size] , axis= [1,2]
            )
        # trường hợp còn lại gán x = shifted_x 
        else:
            x = shifted_x
        
        # Thay đổi lại hình dạng x thành [batch_size, height * width , channels] 
        # để có thể áp dụng lớp noron da tầng 
        x = tf.reshape(x , shape=(-1 , height * width , channels))
        # chuẩn hóa x qua lớp DropPath
        x = self.drop_path(x)
        # tạo ra lớp kết nối dư bằng cách công x_kip vs x
        x = x_skip + x
        # gán lại x_skip = x để thực hiện cho lớp kết nối dư tiếp theo 
        x_skip = x
        # thêm lớp norm2 , mlp ,drop_path
        x = self.norm2(x)
        x = self.mlp(x)
        x = self.drop_path(x)
        # cuối cùng thêm 1 lớp kết nối dư và trả về 
        x = x_skip + x
        # output shape = [batch_size , num_patch , hidden_size]
        return x
            

In [ ]:
class PatchExtract(layers.Layer):
    def __init__(self, patch_size, **kwrgs):
        super().__init__(**kwrgs)
        self.patch_size_x = patch_size[0]
        self.patch_size_y = patch_size[1]

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images = images , 
            sizes=(1, self.patch_size_x, self.patch_size_y, 1),
            strides=(1, self.patch_size_x, self.patch_size_y, 1),
            rates=(1, 1, 1, 1),
            padding="VALID", 
        )
        patch_dim = patches.shape[-1]
        patch_num = patches.shape[1]
        return tf.reshape(patches , (batch_size , patch_num * patch_num , patch_dim))
    


class PatchEmbedding(layers.Layer):
    def __init__(self, num_patch , embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_patch = num_patch 
        self.proj = layers.Dense(embed_dim)
        self.pos_embed = layers.Embedding(input_dim=num_patch, output_dim=embed_dim)

    def call(self, patch):
        pos = tf.range(start=0, limit=self.num_patch, delta=1)
        return self.proj(patch) + self.pos_embed(pos)
    


class PatchMerging(tf.keras.layers.Layer):
    def __init__(self, num_patch, embed_dim):
        super().__init__()
        self.num_patch = num_patch
        self.embed_dim = embed_dim
        self.linear_trans = layers.Dense(2 * embed_dim, use_bias=False)

    def call(self, x):
        # x shape  x = [batch_size , num_patch , hiddent_dim]
        height, width = self.num_patch
        _, _, C = x.get_shape().as_list() # c = 64  
        x = tf.reshape(x, shape=(-1, height, width, C))
        # Tiếp theo, hàm call tách x thành bốn tensor con x0, x1, x2 và x3, 
        # mỗi tensor có kích thước (-1, height // 2, width // 2, C) và chứa các patch ở vị trí
        # chẵn-chẵn, lẻ-chẵn, chẵn-lẻ và lẻ-lẻ của ma trận patch 
        # tức là lấy các chỉ số theo chỉ só hàng và cột với bước nhảy = 2
        x0 = x[:, 0::2, 0::2, :]  # ở đây lấy các giá trị hàng chẵn cột chẵn step 2
        x1 = x[:, 1::2, 0::2, :]  # các giá trị hàng lẻ cột chẵn  step = 2
        x2 = x[:, 0::2, 1::2, :]  # các chỉ số theo hàng chẵn cột lẻ step = 2
        x3 = x[:, 1::2, 1::2, :]  # các chỉ số theo hàng lẻ cột lẻ 
        # Nối các phần lại theo chiều cuối cùng, tức là tăng kích thước đặc trưng lên 4 lần (4C 
        x = tf.concat((x0, x1, x2, x3), axis=-1)
        #  tức là giảm số patch xuống một nửa theo chiều cao và chiều rộng, nhưng tăng kích thước đặc trưng lên gấp đôi
        x = tf.reshape(x, shape=(-1, (height // 2) * (width // 2), 4 * C))
        return self.linear_trans(x)

In [ ]:
input = layers.Input(input_shape)
x = layers.RandomCrop(image_dimension, image_dimension)(input)
x = layers.RandomFlip("horizontal")(x)
x = PatchExtract(patch_size)(x)
x = PatchEmbedding(num_patch_x * num_patch_y, embed_dim)(x)
x = SwinTransformer(
    dim=embed_dim,
    num_patch=(num_patch_x, num_patch_y),
    num_heads=num_heads,
    window_size=window_size,
    shift_size=0,
    num_mlp=num_mlp,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
)(x)
x = SwinTransformer(
    dim=embed_dim,
    num_patch=(num_patch_x, num_patch_y),
    num_heads=num_heads,
    window_size=window_size,
    shift_size=shift_size,
    num_mlp=num_mlp,
    qkv_bias=qkv_bias,
    dropout_rate=dropout_rate,
)(x)
x = PatchMerging((num_patch_x, num_patch_y), embed_dim=embed_dim)(x)
x = layers.GlobalAveragePooling1D()(x)
output = layers.Dense(num_classes, activation="softmax")(x)

In [ ]:
model = keras.Model(input, output)
model.compile(
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing),
    optimizer=tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    ),
    metrics=[
        keras.metrics.CategoricalAccuracy(name="accuracy"),
        keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),
    ],
)

history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_split=validation_split,
)

In [ ]:
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Train and Validation Losses Over Epochs", fontsize=14)
plt.legend()
plt.grid()
plt.show()

In [ ]:
loss, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {round(loss, 2)}")
print(f"Test accuracy: {round(accuracy * 100, 2)}%")
print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")